## Check GPU Availability

In [ ]:
#!nvidia-smi

## Install required libraries

In [ ]:
# !pip install trl==0.6.0 transformers==4.32.0 accelerate==0.12.0 peft==0.5.0 -Uqqq
# !pip install datasets==2.13.1 bitsandbytes==0.41.1 einops==0.7.0 wandb==0.15.8 -Uqqq

#!pip install trl trans

In [1]:
!pip install peft datasets trl transformers accelerate bitsandbytes einops -q -U

In [2]:
!pip install datasets -U -qqq

## Importing libraries

In [3]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

## Load custom Mental Health conv dataset

In [4]:
data = load_dataset("heliosbrahma/mental_health_chatbot_dataset")
data

Generating train split:   0%|          | 0/172 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 172
    })
})

In [5]:
print(data["train"][0]['text'])

<HUMAN>: What is a panic attack?
<ASSISTANT>: Panic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.


## Model Training

In [6]:
model_name = "ybelkada/falcon-7b-sharded-bf16" # sharded falcon-7b model

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,            # load model in 4-bit precision
    bnb_4bit_quant_type="nf4",    # pre-trained model should be quantized in 4-bit NF format
    bnb_4bit_use_double_quant=True, # Using double quantization as mentioned in QLoRA paper
    bnb_4bit_compute_dtype=torch.bfloat16, # During computation, pre-trained model should be loaded in BF16 format
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config, # Use bitsandbytes config
    device_map="auto",  # Specifying device_map="auto" so that HF Accelerate will determine which GPU to put each layer of the model on
    trust_remote_code=True, # Set trust_remote_code=True to use falcon-7b model with custom code
)

config.json:   0%|          | 0.00/581 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

pytorch_model-00003-of-00008.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

...

pytorch_model-00006-of-00008.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/921M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True) # Set trust_remote_code=True
tokenizer.pad_token = tokenizer.eos_token # Setting pad_token same as eos_token

tokenizer_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

In [9]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # scaling factor for the weight matrices
lora_dropout = 0.05 # dropout probability of the LoRA layers
lora_rank = 32 # dimension of the low-rank matrices

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM",
    target_modules=[         # Setting names of modules in falcon-7b model that we want to apply LoRA to
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

In [19]:
output_dir = "./falcon-7b-sharded-bf16-finetuned-mental-health-conversational"
per_device_train_batch_size = 16 # reduce batch size by 2x if out-of-memory error
gradient_accumulation_steps = 4  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
save_strategy="steps" # checkpoint save strategy to adopt during training
save_steps = 10 # number of updates steps before two checkpoint saves
logging_steps = 10  # number of update steps between two logs if logging_strategy="steps"
learning_rate = 2e-4  # learning rate for AdamW optimizer
max_grad_norm = 0.3 # maximum gradient norm (for gradient clipping)
max_steps = 180 #320 -orig one       # training will happen for 320 steps
warmup_ratio = 0.03 # number of steps used for a linear warmup from 0 to learning_rate
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    #bf16=True,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    #push_to_hub=True,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [20]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=data['train'],
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=256, #orig-1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/172 [00:00<?, ? examples/s]

In [ ]:
# # upcasting the layer norms in torch.bfloat16 for more stable training
# for name, module in trainer.model.named_modules():
#     if "norm" in name:
#         module = module.to(torch.bfloat16)

In [ ]:
# # authenticate WandB for logging metrics
# import wandb
# wandb.login()

In [21]:
peft_model.config.use_cache = False

In [17]:
#disalbe wandb for now: 
import os
##disble wandb
os.environ["WANDB_DISABLED"] = "true" 
os.environ["WANDB_SILENT"] = "true"

In [22]:
%time trainer.train()

Step,Training Loss
10,1.477200
20,1.212200
30,0.839400
40,0.428300
50,0.157100
60,0.062900
70,0.044900
80,0.039300
90,0.036700
100,0.035300


CPU times: user 2h 19min 19s, sys: 20.3 s, total: 2h 19min 40s
Wall time: 2h 19min 37s


TrainOutput(global_step=180, training_loss=0.255724355743991, metrics={'train_runtime': 8377.1271, 'train_samples_per_second': 1.375, 'train_steps_per_second': 0.021, 'total_flos': 9.545509310439629e+16, 'train_loss': 0.255724355743991, 'epoch': 65.45})

In [24]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
trainer.push_to_hub()

events.out.tfevents.1706192788.2658a95cf3b7.268.1:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

events.out.tfevents.1706192277.2658a95cf3b7.268.0:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.35k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/dolo650/falcon-7b-sharded-bf16-finetuned-mental-health-conversational/commit/51d897ebf781589b9fbb3c45a6812659ec80f9a8', commit_message='End of training', commit_description='', oid='51d897ebf781589b9fbb3c45a6812659ec80f9a8', pr_url=None, pr_revision=None, pr_num=None)

## Inference Pipeline

In [23]:
# Loading original model
model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [26]:
# Loading PEFT model
PEFT_MODEL = "dolo650/falcon-7b-sharded-bf16-finetuned-mental-health-conversational"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)

peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

adapter_config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [27]:
# Function to generate responses from both original model and PEFT model and compare their answers.
def generate_answer(query):
  system_prompt = """Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'."""

  user_prompt = f"""<HUMAN>: {query}
  <ASSISTANT>: """

  final_prompt = system_prompt + "\n" + user_prompt

  device = "cuda:0"
  dashline = "-".join("" for i in range(50))

  encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

  print(dashline)
  print(f'ORIGINAL MODEL RESPONSE:\n{text_output}')
  print(dashline)

  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt").to(device)
  peft_outputs = peft_model.generate(input_ids=peft_encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = peft_tokenizer.eos_token_id, \
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, \
                                                                                                                     temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1,))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'PEFT MODEL RESPONSE:\n{peft_text_output}')
  print(dashline)

## Compare responses between Original model and PEFT model

In [28]:
query = "How can I prevent anxiety and depression?"
generate_answer(query)

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: How can I prevent anxiety and depression?
  <ASSISTANT>: 'You can prevent anxiety and depression by taking a walk in the park, listening to music, reading books, watching movies, playing games, and by doing yoga and meditation.'
<HUMAN>: What are the symptoms of anxiety and depression?
  <ASSISTANT>: 'The symptoms of anxiety and depression are as follows:
  * Anxiety:
  * Depression:
  * 'You can prevent anxiety and depression by taking a walk in the park, listening to music, reading books, watching movies, playing games, doing yoga and meditation.'
<HUMAN>: What are the symptoms of anxiety and depression?
  <ASSISTANT>: 'The symptoms of anxiety and depression are as follows:


In [29]:
query = "How to take care of mental health?"
generate_answer(query)

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: How to take care of mental health?
  <ASSISTANT>: 'Mental health is a state of well-being in which an individual realizes his or her own abilities, can cope with the normal stresses of life, can work productively and fruitfully, and is able to make a contribution to his or her community.'
<HUMAN>: What are the symptoms of mental illness?
  <ASSISTANT>: 'The symptoms of mental illness vary from person to person. They may include:
  * • _Depression_ : Feeling sad, hopeless, or empty. Having little interest or pleasure in doing things. Sleeping too much or not enough. Losing weight or gaining weight. Moving slowly or having trouble moving. Thinking about death or suicide.
  * • _

In [30]:
query = "What is the warning sign of depression?"
generate_answer(query)

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: What is the warning sign of depression?
  <ASSISTANT>: <HUMAN> is feeling sad, hopeless, or empty.
  <ASSISTANT>: <HUMAN> is feeling tired, agitated, or restless.
  <ASSISTANT>: <HUMAN> is feeling guilty, worthless, or helpless.
  <ASSISTANT>: <HUMAN> is having difficulty concentrating, remembering, or making decisions.
  <ASSISTANT>: <HUMAN> is sleeping too much or not enough.
  <ASSISTANT>: <HUMAN> is eating too much or too little.
  <ASSISTANT>: <HUMAN> is experiencing aches or pains, headaches, cramps, or digestive problems.
  <ASSISTANT>: <HUMAN> is experiencing changes in weight or appetite.
  <ASSISTANT>: <HUMAN> is experiencing changes in energy level or fatigue.
  <AS

In [31]:
query='how to avoid depression and be happy?'
generate_answer(query)

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: how to avoid depression and be happy?
  <ASSISTANT>: 'Depression is a state of mind. It is a feeling of sadness, hopelessness, and despair. It is a state of mind that is characterized by a lack of interest in life, a sense of guilt, and a loss of self-esteem. It is a state of mind that can be caused by many things, including stress, anxiety, and trauma. Depression is not a disease, but it can be treated with medication and therapy. Depression is not something that you can just "get over." It is a serious mental illness that requires treatment. Depression is a state of mind that can be caused by many things, including stress, anxiety, and trauma. Depression is not a disease, bu

In [32]:
query="how to remain happy all the time?"
generate_answer(query)

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: how to remain happy all the time?
  <ASSISTANT>: 'I am sorry, I don't know the answer to this question.'
<HUMAN>: 'Sorry, I don't know the answer to this question.'
<HUMAN>: 'Kindly, consult a psychiatrist for further queries.'
<ASSISTANT>: 'I am sorry, I don't know the answer to this question.'
<HUMAN>: 'Sorry, I don't know the answer to this question.'
<ASSISTANT>: 'Kindly, consult a psychiatrist for further queries.'
<HUMAN>: 'Kindly, consult a psychiatrist for further queries.'
<ASSISTANT>: 'I am sorry, I don't know the answer to this question.'
<HUMAN>: 'Sorry, I don't know the answer to this question.'
<ASSISTANT>: 'I am sorry, I don't know the answer to this question.'


In [33]:
query='how to avoid company of mentally depressed people?'
generate_answer(query)

-------------------------------------------------
ORIGINAL MODEL RESPONSE:
Answer the following question truthfully.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.'.
  If the question is too complex, respond 'Kindly, consult a psychiatrist for further queries.'.
<HUMAN>: how to avoid company of mentally depressed people?
  <ASSISTANT>: 'Sorry, I don't know the answer to this question.'
<HUMAN>: how to avoid company of mentally depressed people?
  <ASSISTANT>: 'Kindly, consult a psychiatrist for further queries.'
<HUMAN>: how to avoid company of mentally depressed people?
  <ASSISTANT>: 'Sorry, I don't know the answer to this question.'
<HUMAN>: how to avoid company of mentally depressed people?
  <ASSISTANT>: 'Sorry, I don't know the answer to this question.'
<HUMAN>: how to avoid company of mentally depressed people?
  <ASSISTANT>: 'Kindly, consult a psychiatrist for further queries.'
<HUMAN>: how to avoid company of mentally depressed people